# Deploying a Model

Once the experiments have been run and a suitable model has been developed, the next step is to create an inference Skill that uses our model. This notebook demonstrates how to do this.

## Selecting the Model

In this example, the model is selected from the last run done in the [set up notebook](00-setup.ipynb). 

In [ ]:
%run ./00-setup.ipynb

In [ ]:
from cortex import Cortex, Message

client = Cortex.client()
exp = client.experiment('flowers/experiment')

exp

After running the cell above, the experiment should display some runs from the previous notebook. The next cell selects the last of these and uses that as the model for the skill in this example.

In [ ]:
run = exp.last_run() 

model = run.get_artifact('model')

print('Run id: %s \nModel name: "%s"' % (run.id, model.name))

## Creating an Action using the Model

An action is built from the model. This is done with a builder, supplied by the Cortex client. In this example, the action name is `flowers/iris-predict`. A model may depend on other python libraries. To make those librarires available to the action, specify them in the `with_requirements` method. You only need to specify libraries that your model needs that are not provided by the cortex python SDK[ * ](#requirements). Here, requirements are for the keras library and the tensorflow backend. The final `build` command builds a docker image and deploys an Action to run it when invoked. 

In [ ]:
builder = client.builder()
builder.action('flowers/iris-predict')\
        .with_requirements(['tensorflow>=1.12,<2',
                            'keras>=2.2.4,<3'])\
        .from_model(model)\
        .build()

Now instaniate the newly saved action so it can be tested. 

In [ ]:
action = client.action('flowers/iris-predict')
action

Below we test the action by invoking it to ensures the action is ready for use.

In [ ]:
params = {'columns': ['sepal_len','sepal_w','petal_len','petal_w'],
           'values': [[4.9,3.1,1.5,0.2]]}

result = action.invoke(message=client.message(params))
print(result.payload)

In the cell below, the maximum value for the three node output of the neural network is selected and matched to the appropriate type of iris flower. 

In [ ]:
x = numpy.argmax(result.payload['values'][0])

iris_dict = {0:'Iris-setosa',1:'Iris-versicolor',2:'Iris-virginica'}

iris_dict[x]

## Creating a Skill using the Action

To create a skill from the action, the builder is called to get a skill builder. 

In [ ]:
skill_builder = builder.skill('flowers/iris-predict')\
     .title('Iris Flower Prediction')\
     .description('Measurement based prediction for the type of Iris flowers.')


Next a schema is built, and then used by the skill builder to identify elements of the input. In this case, the schema describes the measurement values for parts of the Iris flower. 

In [ ]:
schema_name = 'flowers/iris-input-schema'

builder.schema(schema_name)\
     .title('Iris Prediction Schema Instance')\
     .from_parameters([{'name': 'values', 'type': 'array', 'format': 'float32'}]).build()

skill_builder = skill_builder.input('iris-measurement').title('Iris Measurements')\
     .use_schema(schema_name)\
     .all_routing(action, 'measurement-based-prediction').build()

Finally, specify the skill construction, a description of the skill is provided, including a name that matches the action name, and also a description of the output.

In [ ]:
skill_builder = skill_builder.output('measurement-based-prediction').title('Classification')\
    .parameter(name='IrisClass', type='array', format='float32').build()

The skill builder is fully specified. Call `to_camel` so the contents can be reviewed.

In [ ]:
skill_builder.to_camel()

## Publish the Skill

Publishing the skill is accomplished by invoking the builder. Once published it can be used in Cortex Studio with other skills to compose agents. 

In [ ]:
skill = skill_builder.build()
print('%s (%s) v%d' % (skill.title, skill.name, skill.version))

----

<a id='requirements'>*</a> Cortex Python SDK Provided Requirements 

These are the requirements that are provided by the Cortex Python SDK and need not be added to an action Docker Image. 

* seaborn>=0.9.0,<0.10
* dill==0.2.8.2
* discovery-transitioning-utils>=1.3.50,<2
* fdk==0.0.31
* dataclasses>=0.6
* seaborn>=0.9.0,<0.10
* matplotlib>=2.2.2,<3
* more_itertools>=4.3.0,<5
* pyyaml>=3.13,<4
* cuid>=0.3,<1
* maya==0.5.0
* docker==3.5.0
* deprecation==2.0.6
* tenacity==5.0.2